In [1]:
import os, operator, nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import networkx as nx

In [2]:
dataset_dir = 'Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/'
    
for subdir, dirs, files in os.walk(dataset_dir):
    # iterate over all files in train/test directory
    for file in files:
        if not file.endswith('.txt'):
            continue
        f = open(subdir+'/'+file, 'r')
        text = f.read()
        #print file
        # tokenizing the text and filtering nouns / verbs etc
        text = text.decode('utf-8')
        tokens = nltk.word_tokenize(text)
        tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

        def filter_words(key):
            return True if key.startswith('NN') or key.startswith('JJ') or key.startswith('VB') else False
        filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))
        
        # creating local maps for nodes and edges
        dict_text = defaultdict(int)
        for t in filt_text:
            dict_text[t] += 1
        edges, nodes, nodes_number = [], {}, {}
        c = 0
        for d in dict_text:
            if d in nodes:
                continue
            nodes[d] = c
            nodes_number[c] = d
            c += 1
            
        # create edges for graph-of-words
        k = 10 ## window size
        for i in range(len(filt_text)-k):
            for j in range(i+1, i+k+1):
                wt = j-i  ## this is the weight of the edge (distance between words)
                edges.append((nodes[filt_text[i]], nodes[filt_text[j]], wt))
                
        #  Write these edges to a file
        

In [4]:
f = open('Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/train/art_and_culture-20893614.txt', 'r')
text = f.read()

In [5]:
# tokenizing the text and filtering nouns / verbs etc
tokens = nltk.word_tokenize(text)
tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

def filter_words(key):
    return True if key.startswith('NN') or key.startswith('JJ') or key.startswith('VB') else False
filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))

In [6]:
# creating local maps for nodes and edges
dict_text = defaultdict(int)
for t in filt_text:
    dict_text[t] += 1
edges = []
nodes = {}
nodes_number = {}
c = 0
for d in dict_text:
    if d in nodes:
        continue
    nodes[d] = c
    nodes_number[c] = d
    c += 1

In [7]:
# create edges for graph-of-words
k = 5 ## window size
for i in range(len(filt_text)-k):
    for j in range(i+1, i+k+1):
        wt = j-i  ## this is the weight of the edge (distance between words)
        edges.append((nodes[filt_text[i]], nodes[filt_text[j]], wt))

In [8]:
# create a file and store the edges
#fw = open('Dataset/500N-KPCrowd-Edges/train/art_and_culture-20893614.txt', 'w')
#fw.write(edges)

In [9]:
# create graph-of-words
G=nx.Graph()

### TODO: read from file to add edges
G.add_weighted_edges_from(edges)

In [72]:
def computeMetric(origKeys, predKeys):
    n_hits = 0
    n_misses = 0
    n_false_alarms = 0
    for k in predKeys:
        if k in origKeys:
            n_hits += 1
        else:
            n_false_alarms += 1
    for k in origKeys:
        if k not in predKeys:
            n_misses += 1
  
    precision = n_hits*1.0/(n_hits+n_false_alarms)
    recall = n_hits*1.0/(n_hits+n_misses)
  
    F1_score = 2.0*(precision*recall)/(precision+recall)

    print ("Precision = " + str(precision))
    print ("Recall = " + str(recall))
    print ("F1 score = " + str(F1_score))
    #return(list(precision, recall, F1_score))    

def combinePhrases(initKeys, split_text):
    result = []
    len_text = len(split_text)
    for i in xrange(len_text):
        word = split_text[i]
        if word in initKeys:
            combined_word = [word]
            if i + 1 == len_text: result.append(word)   # appends last word if keyword and doesn't iterate
            for j in xrange(i + 1, len_text):
                other_word = split_text[j]
                if other_word in initKeys:
                    combined_word.append(other_word)
                else:
                    for w in combined_word: 
                        if w in initKeys:
                            initKeys.remove(w)
                    result.append(" ".join(combined_word))
                    break
    return result

In [76]:
f = open('../train/art_and_culture-20893614.key', 'r')
origKeys = f.read().split("\n")

# extracting keywords using simple degree centrality
centrality = nx.degree_centrality(G)
sorted_centr = sorted(centrality, key=centrality.get, reverse=True)
initKeys = [nodes_number[i] for i in sorted_centr]
predKeys = combinePhrases(initKeys[:int(0.33*len(initKeys))], filt_text)
computeMetric(origKeys, predKeys)

Precision = 0.260869565217
Recall = 0.142857142857
F1 score = 0.184615384615


In [77]:
# extracting keywords using page-rank
page_ranks = nx.pagerank(G)
sorted_page_ranks = sorted(page_ranks, key=page_ranks.get, reverse=True)
initKeys = [nodes_number[i] for i in sorted_page_ranks[:int(0.33*len(sorted_centr))]]
predKeys = combinePhrases(initKeys[:int(0.33*len(initKeys))], filt_text)
computeMetric(origKeys, predKeys)
#for i in sorted_page_ranks[:30]:
#    print nodes_number[i]

Precision = 0.222222222222
Recall = 0.047619047619
F1 score = 0.078431372549


## Text Rank

In [14]:
edges=[]

In [15]:
wt={}
# create edges for graph-of-words
k = 5 ## window size
for i in range(len(filt_text)-k):
    for j in range(i+1, i+k+1):
        wt[(nodes[filt_text[i]],nodes[filt_text[j]])] = j-i 
        wt[(nodes[filt_text[j]],nodes[filt_text[i]])] = j-i## this is the weight of the edge (distance between words)
        edges.append((nodes[filt_text[i]], nodes[filt_text[j]], j-i))

In [16]:
G=nx.Graph()
### TODO: read from file to add edges
G.add_weighted_edges_from(edges)

In [17]:
#G.in_edges(0)

In [18]:
#G.out_edges(0)

In [19]:
#directed graph
# r={}
# for i in G.nodes():
#     r[i]=1
# d=0.85 #damping factor
# t1= 1-d
# for itr in range(20):
#     for i in G.nodes():
#         for j in G.in_edges(i):
#                 t2 = wt[(j[0],j[1])]*r[j[0]]*1.0
#                 wk=0
#                 for k in G.out_edges(j):
#                     wk += wt[(k[0],k[1])] 
#                     t2=d*(t2*1.0/wk)
#         r[i]=t1+t2

In [20]:
#undirected graph
r={}
for i in G.nodes():
    r[i]=1
d=0.85 #damping factor
t1= 1-d
for itr in range(20):
    for i in G.nodes():
        for j in G.neighbors(i):
                t2 = wt[(j,i)]*r[j]*1.0
                wk=0
                for k in G.neighbors(j):
                    wk += wt[(k,j)] 
                    t2=d*(t2*1.0/wk)
        r[i]=t1+t2

In [21]:
sorted_r = sorted(r.items(), key=operator.itemgetter(1),reverse=True)

In [22]:
for k,v in sorted_r[:30]:
    print nodes_number[k]

messages
day
sent
March
AIRES
BUENOS
Photo/Natacha
lilac
mansion
Twitter
tour
chiffon
Facebook
involved
followers
wore
dress
Canada
tennis
bouquet
rice
homes
Vancouver
America
Monaco
player
years
purple
threw
North


In [26]:
predKeys = [nodes_number[i] for i in sorted_r[:int(0.33*len(sorted_r))]]

KeyError: (54, 0.16505208333337462)

## Text rank using a github 'summa' package

In [24]:
#key phrases
from summa import keywords
predKeys = keywords.keywords(" ".join(filt_text))
#predKeys = [nodes_number[i] for i in keywords[:int(0.33*len(sorted_centr))]]
#computeMetric(origKeys, predKeys)

In [25]:
print predKeys

lopilato
argentine
buble weds
fans
buenos
civil wedding ceremony
concert
aires
married
pop
actress
love
wore
ap
thursday
tv star
michael
beautiful
couple
north


## TF-IDF

In [89]:
from __future__ import division, unicode_literals 

In [90]:
dataset_dir = 'Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/'
documents=[]
for subdir, dirs, files in os.walk(dataset_dir):
    # iterate over all files in train/test directory
    for file in files:
        if not file.endswith('.txt'):
            continue
        f = open(subdir+'/'+file, 'r')
        text = f.read()
        print file
        # tokenizing the text and filtering nouns / verbs etc
        text = text.decode('utf-8')
        tokens = nltk.word_tokenize(text)
        tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

        def filter_words(key):
            return True if key.startswith('NN') or key.startswith('JJ') or key.startswith('VB') else False
        filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))
        documents.append(filt_text)
        


art_and_culture-20880868-justTitle.txt
art_and_culture-20881913-justTitle.txt
art_and_culture-20885615-justTitle.txt
art_and_culture-20886831-justTitle.txt
art_and_culture-20890475-justTitle.txt
business-20838979-justTitle.txt
business-20871890-justTitle.txt
business-20884299-justTitle.txt
business-20891485-justTitle.txt
business-20903055-justTitle.txt
crime-20905140-justTitle.txt
crime-20906537-justTitle.txt
crime-20907926-justTitle.txt
crime-20912842-justTitle.txt
crime-20914930-justTitle.txt
fashion-20815941-justTitle.txt
fashion-20837495-justTitle.txt
fashion-20841639-justTitle.txt
fashion-20848421-justTitle.txt
fashion-20857483-justTitle.txt
health-20858223-justTitle.txt
health-20871579-justTitle.txt
health-20876242-justTitle.txt
health-20881535-justTitle.txt
health-20892135-justTitle.txt
politics_us-20771981-justTitle.txt
politics_us-20782177-justTitle.txt
politics_us-20802905-justTitle.txt
politics_us-20804565-justTitle.txt
politics_us-20805637-justTitle.txt
politics_world-20779

In [91]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [92]:
documents[:10]

[[u'Phish',
  u'Phavors',
  u'New',
  u'York',
  u'3-Day',
  u'Phestival',
  u'Phinger',
  u'Lakes'],
 [u'Movie', u'Review', u'A', u'Better', u'World'],
 [u'True',
  u'Grit',
  u'star',
  u'Hailee',
  u'Steinfeld',
  u'talks',
  u'play',
  u'Sleeping',
  u'Beauty'],
 [u'lsquo', u'Miral', u'rsquo', u'Director', u'conflict', u'interest'],
 [u'Depp', u'back', u'Pirates'],
 [u'H',
  u'amp',
  u'M',
  u'Profit',
  u'Misses',
  u'Estimates',
  u'Retailer',
  u'Absorbs',
  u'Cost',
  u'Rises'],
 [u'Audio',
  u'Reveals',
  u'Rajaratnam',
  u'Frantic',
  u'Calls',
  u'Goldman-Buffett',
  u'Tip'],
 [u'GM', u'sells', u'Delphi', u'stake'],
 [u'Google', u'faces', u'tax', u'investigation', u'China'],
 [u'Alaska', u'oil', u'bill', u'draws', u'early', u'fire']]

In [93]:
bloblist = [tb(" ".join(d)) for d in documents]

In [94]:
bloblist[:10]

[TextBlob("Phish Phavors New York 3-Day Phestival Phinger Lakes"),
 TextBlob("Movie Review A Better World"),
 TextBlob("True Grit star Hailee Steinfeld talks play Sleeping Beauty"),
 TextBlob("lsquo Miral rsquo Director conflict interest"),
 TextBlob("Depp back Pirates"),
 TextBlob("H amp M Profit Misses Estimates Retailer Absorbs Cost Rises"),
 TextBlob("Audio Reveals Rajaratnam Frantic Calls Goldman-Buffett Tip"),
 TextBlob("GM sells Delphi stake"),
 TextBlob("Google faces tax investigation China"),
 TextBlob("Alaska oil bill draws early fire")]

In [95]:
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:5]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1
	Word: 3-Day, TF-IDF: 0.77041
	Word: Phavors, TF-IDF: 0.77041
	Word: Lakes, TF-IDF: 0.77041
	Word: Phestival, TF-IDF: 0.77041
	Word: Phinger, TF-IDF: 0.77041
Top words in document 2
	Word: Better, TF-IDF: 1.15157
	Word: Review, TF-IDF: 0.98211
	Word: Movie, TF-IDF: 0.7824
	Word: World, TF-IDF: 0.65459
	Word: A, TF-IDF: 0.54912
Top words in document 3
	Word: Grit, TF-IDF: 0.68481
	Word: Sleeping, TF-IDF: 0.68481
	Word: Steinfeld, TF-IDF: 0.68481
	Word: Hailee, TF-IDF: 0.68481
	Word: Beauty, TF-IDF: 0.583
Top words in document 4
	Word: Miral, TF-IDF: 1.02722
	Word: lsquo, TF-IDF: 0.91169
	Word: Director, TF-IDF: 0.81843
	Word: rsquo, TF-IDF: 0.79617
	Word: conflict, TF-IDF: 0.6914
Top words in document 5
	Word: back, TF-IDF: 1.74901
	Word: Depp, TF-IDF: 1.55308
	Word: Pirates, TF-IDF: 1.48619
Top words in document 6
	Word: Profit, TF-IDF: 0.61633
	Word: Retailer, TF-IDF: 0.61633
	Word: Misses, TF-IDF: 0.61633
	Word: Absorbs, TF-IDF: 0.61633
	Word: Rises, TF-IDF: 0

KeyboardInterrupt: 

Top words in document 52  
	Word: Lopilato, TF-IDF: 0.26024  
	Word: Buble, TF-IDF: 0.19518  
	Word: Buenos, TF-IDF: 0.1741  
	Word: AIRES, TF-IDF: 0.1741  
	Word: BUENOS, TF-IDF: 0.1741  

## TF-IDF  using scikit

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

In [86]:
dataset_dir = '../'
token_dict={}
for subdir, dirs, files in os.walk(dataset_dir):
    # iterate over all files in train/test directory
    for file in files:
        if not file.endswith('.txt'):
            continue
        f = open(subdir+'/'+file, 'r')
        text = f.read()
        #print file
        # tokenizing the text and filtering nouns / verbs etc
        text = text.decode('utf-8')
        tokens=tokenize(text)
        tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

        def filter_words(key):
            return True if key.startswith('NN') or key.startswith('JJ') or key.startswith('VB') else False
        filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))
        token_dict[file]=" ".join(filt_text)
        


In [87]:
tfidf = TfidfVectorizer(tokenizer= tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

In [88]:
dense = tfs.todense()

In [89]:
doc = dense[5].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]

In [92]:
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:10]:
   print('{0: <20} {1}'.format(phrase, score))

NameError: name 'feature_names' is not defined